# Title

#### 1. Student name: Qingsheng Long
#### 1. Student ID: 16387388

#### 2. Student name:
#### 2. Student ID: 

# Introduction

Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum


## List the key steps you took to perform EDA

1. 使用官方提供的interpolate_and_split.py文件对train_raw数据进行处理，得到新的训练数据集train_2.csv
2. 使用merge_training_data.py对官方提供的train.csv和train_2.csv进行合并，并针对数据进行去重，得到更大的训练数据集train_merged.csv
3. 把训练集进行分割为X_train和y_train两部分，用于训练模型
4. 使用自定义函数def clean_input(X)对数据进行空值处理，主要采用bfill和ffill
5. 使用K-flod对训练集交叉验证，并计算模型MSE


## List and describe your kNN methodological design you used for experimental testing, benchmarking against a baseline, tuning, ensuring data leakage was not ocurring and how you effectively summarised and communicated all your results

1. 本次分析/预测主要采用sklearn.neighbors包中的KNeighborsRegressor模型进行
2. 在模型的预测中，训练集的-299~0列作为输入特征，1~300列作为输出标签
3. 同时使用自定义函数find_best_parameter(max_k, metric_list, X_train, y_train, n_splits=5)对模型进行K-fold交叉验证，以求寻找到最佳的k-metric组合
4. 在K-fold交叉验证过程中，针对k = 1~100, metric = ['euclidean', 'manhattan', 'cosine']共计300种参数组合进行验证
5. 明确划分train.csv用于训练，test.csv用于预测，同时使用StandardScaler对数据进行归一化

###  Best Kaggle score submission based on kNN

- STATE HERE YOUR BEST SCORE AND WHERE YOU RANKED IN THE CLASS AT THE TIME OF SUBMISSION

- EMBED THE SCREENSHOT OF YOUR BEST SUBMISSION SCORE FROM KAGGLE HERE


## List all the predictive algorithms you used for the classification modelling component where you had the freedom to explore alternative methods. List here also your best Kaggle score and your position on the Kaggle leaderboard

1. KNeighborsRegressor
2. MLPRegressor
3. ...
4. ...



## Analysis: list here all the remaining test design strategies you used, like k-fold CV, hyperparameter tuning, feature selection and feature engineering, discussion of your submission score results over time with resepct to model refinements and their effects

1. K-Fold Cross-Validation：本实验中使用自定义函数find_best_parameter进行K-Fold Cross-Validation，在KNN模型中，评估不同的k值与metric对MSE的影响，从而选择出最佳的k-metric组合。在MLP模型中，则是对hyperparameters如hidden_layer_sizes，activations以及alphas进行测试，以求选择一个泛化能力良好的配置
2. Hyperparameter Tuning：在KNN模型中，主要调整了k值(从1~100)和distance metrics(['euclidean', 'manhattan', 'cosine'])，并在K-Fold CV中检验模型MSE数值。而在MLP模型中，我采用了两中不同的策略，其中一种是：使用K-Fold CV找到最佳的参数组合，然后利用完整的训练集进行训练；另外一种则是不采用K-Fold CV，采用固定的参数搭配，利用train_test_split划分训练集来训练模型。
3. Feature Engineering：因为训练数据中存在大量空值，而官方数据说明中表示"values for columns -49..0 are present in all the rows"，因此我分别尝试了采用[-299, 0]和]-49, 0]作为输入标签来训练模型。
4. ...




## Bonus material

### If you performed cluster analysis, decribe below what you did, how you did and what was the meaning and output:
1. ...
2. ...

### If you conducted extensive extraction of additional features, list them below and explain how you extracted them and if each one was useful and how you determined this:

1. ...
2. ...
3. ...
4. ...


## Did you submit your initial kaggle solution by May 1 and enter it into the Google Spreadsheet

- YES

## Have you submitted your reading log and integrated insights from your reading into this assignment

- YES / NO
- Discuss here how has your reading informed and helped guide your assignment 3? Which papers were particularly relevant and helpful and why? 

## Have you submitted your AI Use Statement

- YES / NO

  
### Executive Summary

Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum


In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import KFold
from datetime import datetime
from submission_record import prepare_submission

# EDA

In [9]:
def clean_input(X):
    return pd.DataFrame(X).interpolate(axis=1).bfill(axis=1).ffill(axis=1).to_numpy()

In [17]:
DATA_DIR = "../data"
train_dataset = "train.csv"  # use the original dataset train.csv
# train_dataset = "train_raw.csv"  # use the dataset train.csv which is built by 'interpolate_and_split.py'
# train_dataset = "train_merged.csv"  # use the dataset which merge train.csv and train_2.csv
train_df = pd.read_csv(os.path.join(DATA_DIR, train_dataset))
test_df = pd.read_csv(os.path.join(DATA_DIR, "test.csv"))
X_test = test_df.loc[:, [str(i) for i in range(-299, 1)]].to_numpy(dtype=np.float32)

In [13]:
X_train = train_df.loc[:, [str(i) for i in range(-299, 1)]].to_numpy(dtype=np.float32)
y_train = train_df.loc[:, [str(i) for i in range(1, 301)]].to_numpy(dtype=np.float32)

X_train2 = train_df.loc[:, [str(i) for i in range(1, 301)]].to_numpy(dtype=np.float32)
y_train2 = train_df.loc[:, [f"r_{r}_pos_{i}" for r in range (1, 10) for i in range(1, 301)]].to_numpy(dtype=np.float32)

In [15]:
X_train = clean_input(X_train)
y_train = clean_input(y_train)

X_train2 = clean_input(X_train2)
y_train2 = clean_input(y_train2)

In [25]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

scaler2 = StandardScaler()
X_train2 = scaler2.fit_transform(X_train2)

X_test = clean_input(X_test)
X_test = scaler.transform(X_test)

# kNN Predictive Modelling


In [30]:
def find_best_parameter(max_k, metric_list, X_train, y_train, n_splits=5):
    result_list = []
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    print("Looking for the parameter combination with the smallest MSE...")
    for k in range(1, max_k + 1):
        for metric in metric_list:
            mse_fold = []
            for train_index, val_index in kf.split(X_train):
                X_tr, X_val = X_train[train_index], X_train[val_index]
                y_tr, y_val = y_train[train_index], y_train[val_index]
                model = KNeighborsRegressor(n_neighbors=k, weights='distance', metric=metric)
                model.fit(X_tr, y_tr)
                y_pred = model.predict(X_val)
                mse = mean_squared_error(y_val, y_pred)
                mse_fold.append(mse)

            avg_mse = np.mean(mse_fold)
            result_list.append((metric, k, avg_mse))

    # Find the combination with the smallest MSE
    best_result = min(result_list, key=lambda x: x[2])
    print(f"\nBest Parameter: metric={best_result[0]}, k={best_result[1]}, MSE={best_result[2]:.5f}")

    return best_result

In [ ]:
best_parameter1 = find_best_parameter(100, ['euclidean', 'manhattan', 'cosine'], X_train, y_train)
best_parameter2 = find_best_parameter(100, ['euclidean', 'manhattan', 'cosine'], X_train2, y_train2)

Looking for the parameter combination with the smallest MSE...

Best Parameter: metric=euclidean, k=49, MSE=8.73558
Looking for the parameter combination with the smallest MSE...


In [ ]:
print("Training KNN Model...")
model1 = KNeighborsRegressor(n_neighbors=best_parameter1[1], weights='distance', metric=best_parameter1[0])
model1.fit(X_train, y_train)

model2 = KNeighborsRegressor(n_neighbors=best_parameter2[1], weights='distance', metric=best_parameter2[0])
model2.fit(X_train2, y_train2)

In [ ]:
print("Predicting...")
y_pred = model1.predict(X_test)
X_test2 = scaler2.transform(y_pred)
y_pred_r_1_to_9 = model2.predict(X_test2)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
prepare_submission.prepareSubmission(y_pred, "../submission_record", "knn", y_pred_r_1_to_9)

# Modelling with alternative algorithms 

In [ ]:
import os
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from submission_record import prepare_submission

In [ ]:
# Split dataset for training and prediction
tX, vX, ty, vy = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
tX2, vX2, ty2, vy2 = train_test_split(X_train2, y_train2, test_size=0.2, random_state=42)

In [ ]:
print("Training MLP Model...")
model1 = MLPRegressor(
    hidden_layer_sizes=(512, 256),
    activation='tanh',
    solver='adam',
    learning_rate_init=0.001,
    max_iter=2000,
    random_state=42,
    verbose=True,
    alpha=1e-4
)
model1.fit(tX, ty)

In [ ]:
model2 = MLPRegressor(
    hidden_layer_sizes=(512, 256),
    activation='tanh',
    solver='adam',
    learning_rate_init=0.001,
    max_iter=2000,
    random_state=42,
    verbose=True,
    alpha=1e-4
)
model2.fit(tX2, ty2)

In [ ]:
# Prediction
y_pred = model1.predict(X_test)
X_test2 = scaler2.transform(y_pred)
y_pred_r_1_to_9 = model2.predict(X_test2)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
prepare_submission.prepareSubmission(y_pred, "../../submission_record", "mlp")

# History and trend of Kaggle submission scores

- try to plot your scores over time and discuss what experimental tweaks you performed at each step

# Bonus Material


### Cluster analysis


### Description of how you extracted additional features

# Conclusion 


Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum

## Key findings:

1. Lorem ipsum dolor sit amet
2. Lorem ipsum dolor sit amet
3. Lorem ipsum dolor sit amet
4. ...


## Discuss what you learned the most fdrom this assignment

...

## Bibliography (list of materials/code sources you used or refered to) 